In [22]:
import json 
import pandas as pd
def SydGrid(file_name):
    '''
    "geo":{"type":"Point","coordinates":[-33.86751,151.20797]}
    all 8 acesss and read same time
    datastructures to optimize performance
    showq -q snowy| less
    master has rank 0 always
    sbatch <script_name> 
    squeue -u <username>
    module avail mpip4py//list all libs
    tell module load <lib> in slurm script
    scancel <job_id>
    '''
    syd_grid = []
    with open(file_name) as f:
        data = json.load(f)
        for val in data['features']:
            grid_data = {}
            properties = val['properties']
            grid_data['id'] = properties['id']
            num = int((grid_data['id'])%4)
            if num == 0:
                num=4
            grid_data['cell'] = chr(int(num+64))+str(int((grid_data['id']-1)/4)+1)
            geometry = val['geometry'] 
            grid_data['xmin'] = geometry['coordinates'][0][0][0]
            grid_data['ymin'] = geometry['coordinates'][0][2][1]
            grid_data['xmax'] = geometry['coordinates'][0][1][0]
            grid_data['ymax'] = geometry['coordinates'][0][0][1]
            syd_grid.append(grid_data)
    return syd_grid
grid = SydGrid('./sydGrid.json')
grid

[{'id': 1,
  'cell': 'A1',
  'xmin': 149.79255080532937,
  'ymin': -33.31644989181766,
  'xmax': 150.34255080532938,
  'ymax': -32.81644989181766},
 {'id': 2,
  'cell': 'B1',
  'xmin': 149.79255080532937,
  'ymin': -33.81644989181766,
  'xmax': 150.34255080532938,
  'ymax': -33.31644989181766},
 {'id': 3,
  'cell': 'C1',
  'xmin': 149.79255080532937,
  'ymin': -34.31644989181766,
  'xmax': 150.34255080532938,
  'ymax': -33.81644989181766},
 {'id': 4,
  'cell': 'D1',
  'xmin': 149.79255080532937,
  'ymin': -34.81644989181766,
  'xmax': 150.34255080532938,
  'ymax': -34.31644989181766},
 {'id': 5,
  'cell': 'A2',
  'xmin': 150.34255080532938,
  'ymin': -33.31644989181766,
  'xmax': 150.8925508053294,
  'ymax': -32.81644989181766},
 {'id': 6,
  'cell': 'B2',
  'xmin': 150.34255080532938,
  'ymin': -33.81644989181766,
  'xmax': 150.8925508053294,
  'ymax': -33.31644989181766},
 {'id': 7,
  'cell': 'C2',
  'xmin': 150.34255080532938,
  'ymin': -34.31644989181766,
  'xmax': 150.8925508053294

In [23]:
def LangCodes(file_name):
    '''
    output is kv pair with lang_code:lang
    '''
    lang_codes = dict()
    with open(file_name, encoding='utf-8') as f:
            data = json.load(f) 
            for val in data:
                lang_codes[val['code']] = val['name']
            return lang_codes
lc = LangCodes('./language_codes.json')
print((lc))

{'fr': 'French', 'en': 'English', 'ar': 'Arabic', 'ja': 'Japanese', 'es': 'Spanish', 'de': 'German', 'it': 'Italian', 'in': 'Indonesian', 'pt': 'Portuguese', 'ko': 'Korean', 'tr': 'Turkish', 'ru': 'Russian', 'nl': 'Dutch', 'fil': 'Filipino', 'msa': 'Malay', 'zh': 'Chinese', 'hi': 'Hindi', 'no': 'Norwegian', 'sv': 'Swedish', 'fi': 'Finnish', 'da': 'Danish', 'pl': 'Polish', 'hu': 'Hungarian', 'fa': 'Persian', 'he': 'Hebrew', 'ur': 'Urdu', 'th': 'Thai', 'uk': 'Ukrainian', 'ca': 'Catalan', 'ga': 'Irish', 'el': 'Greek', 'eu': 'Basque', 'cs': 'Czech', 'gl': 'Galician', 'ro': 'Romanian', 'hr': 'Croatian', 'vi': 'Vietnamese', 'bn': 'Bangla', 'bg': 'Bulgarian', 'sr': 'Serbian', 'sk': 'Slovak', 'gu': 'Gujarati', 'mr': 'Marathi', 'ta': 'Tamil', 'kn': 'Kannada'}


In [24]:
def GetData(tweet):
    tweet_data = {}
    if type(tweet['geo'])!= type(None):
        language = tweet['doc']['metadata']['iso_language_code']
        x = tweet['doc']['geo']['coordinates'][1]
        y = tweet['doc']['geo']['coordinates'][0]
        #checking if the point lies in grid
        if 151.992551 < x < 149.792551 and -32.81645 < y < -34.81645:
            tweet_data['language'] = language
            tweet_data['grid_cell'] = GetGridCell(x,y)
    return tweet_data

In [25]:
def total_language_count(dict_lang, lang_codes):
    """Print the results from language code counting.
    Keyword arguments:
    counter_language -- counter with language counts
    supported_languages -- dict mapping lancode to name
    """
    for item in dict_lang:
        print("Cell :",item)
        print('# tweets', sum(dict_lang[item].values()))
        print('# languages :', len(dict_lang[item]))
        counter_language = dict_lang[item]
        count = 1
        print("Most common languages in dataset:", flush=True)
        for language in counter_language.most_common(10):
            #if there is a bug its here
            print(str(count) + ". " + lang_codes[language[0]] + " (" + language[0] + ")" + ", " + str(language[1]), flush=True)
            count += 1

In [26]:
def break_chunks(file_name, chunk_size, total_size):
    with open(file_name,'rb') as f:
        #position pointer for file 
        chunk_end = f.tell()
        while True:
            # we update this for every 
            chunk_start = chunk_end
            # define ending of the chunk
            f.seek(f.tell()+chunk_size)
            # read everyline 
            f.readline()
            #checking if the line is inside the chunk 
            chunk_end = f.tell()
            #getting the size required
            if chunk_end > total_size:
                chunk_end = total_size
            yield chunk_start, chunk_end-chunk_start
            if chunk_end == total_size:
                break

In [27]:
def break_batches(file_path, chunk_start, chunk_size, batch_size):
    with open(file_path, 'rb') as f:
        batch_end = chunk_start

        while True:
            batch_start = batch_end
            # go to current position + batch_size
            f.seek(batch_start + batch_size)
            # read the lines as a whole
            f.readline()
            batch_end = f.tell()
            if batch_end > chunk_start + chunk_size:
                batch_end = chunk_start + chunk_size
            yield batch_start, batch_end - batch_start
            # reaching the end of chunk
            if batch_end == chunk_start + chunk_size:
                break

In [33]:
def GetGridCell(x,y):
    grid = SydGrid('./sydGrid.json')
    for item in grid:
        if item['xmin']<x<=item['xmax'] and item['ymin']<y<=item['ymax']:
            return item['cell']

str

In [ ]:
from collections import Counter

class DataProcessor():

    def __init__(self, batch_size):
        self.batch_size = batch_size #BATCH_SIZE in repo
        self.lang_counter = dict(Counter())
    
    def get_results(self): #retrive_results in repo
        result = {"language": self.lang_counter}
        return result

    def process_tweet(self, tweet):
        """Process tweet and perform counting operations
        Keyword arguments:
        tweet -- tweet in JSON format
        """
        # Extract language
        data = GetData(tweet)
        language = data['language']
        cell = data['grid_cell']
        self.lang_counter[cell][language] += 1
        

    def process_wrapper(self, path_to_dataset, chunk_start, chunk_size):
        """Main method executed by worker process to split chunk into smaller
        batches and process batches sequentially
        Keyword arguments:
        path_to_dataset -- Path to dataset to be split up
        chunk_start -- Byte offset of chunk from beginning of file
        chunk_size -- Size of chunk in bytes
        """
        with open(path_to_dataset, 'rb') as f:
            batches = []

            # Split up chunk into batches of size BATCH_SIZE
            for read_start, read_size in break_batches(path_to_dataset, chunk_start, chunk_size, self.batch_size):
                batches.append({"batchStart": read_start, "batchSize": read_size})

            # Process batches sequentially
            for batch in batches:

                # Move to start position of batch
                f.seek(batch['batchStart'])

                if batch['batchSize'] > 0:
                    # Read in next batch in bytes as given per batchSize and
                    # split lines
                    content = f.read(batch['batchSize']).splitlines()

                    for line in content:
                        # Decode each line as utf-8 string
                        line = line.decode('utf-8')  # Convert to utf-8
                        if line[-1] == ",":  # if line has comma
                            line = line[:-1]  # removing trailing comma
                        try:
                            # Load tweet in JSON format
                            tweet = json.loads(line)
                            self.process_tweet(tweet)

                        except Exception as e:
                            print("Error reading row from JSON file - ignoring")
                            print(line)
                else:
                    print("batchsize with size 0 detected")

In [17]:
import argparse
import os
from datetime import datetime as dt
from mpi4py import MPI

# Get start time for the program
STARTTIME = dt.now()
ENDTIME = None

# Start doing MPI
COMM = MPI.COMM_WORLD
SIZE = COMM.Get_size()
RANK = COMM.Get_rank()

dataset = 'smallTwitter.json'
langcodes = 'language_codes.json'
sydGrid = "sydGrid.json"

"""
argParser = argparse.ArgumentParser()
argParser.add_argument('--dataset', type = str, default = 'bigTwitter.json')
argParser.add_argument('--langcodes', type = str, default = 'language_codes.json')
args = argParser.parse_args()
"""

dataSetPath = "./" + dataset
codesPath = "./" + langcodes
gridPath = "./" + sydGrid

lc = LangCodes(codesPath)
grid = SydGrid(gridPath)

def main():
    dataProcessor = DataProcessor()
    if RANK == 0:
        dataTotSize = os.path.getsize(dataSetPath)
        sizePerProcess = dataTotSize / SIZE
        
        chunks = []
        for chunkStart, chunkSize in break_chunks(dataSetPath,
                                              int(sizePerProcess),
                                              dataTotSize):
            chunks.append({"chunkStart": chunkStart, "chunkSize": chunkSize})
    else:
        chunks = None
        
    COMM.Barrier()
    
    ## Still waiting on helper functions to be finished and defined before we can fill in blanks
    
    processorChunks = COMM.scatter(chunks, root=0)
    
    if RANK != 0:
        ENDTIME = dt.now()
        
    COMM.Barrier()
    
    ## Still waiting on helper functions to be finished and defined before we can fill in blanks
    
if __name__ == "__main__":
    main()

In [50]:
from collections import Counter
f =dict(Counter())
a='hi'
f['hi'] = Counter('hello')
f['bi'] = Counter('bello')
cntr = f[a]
for language in cntr.most_common(2):
    print(language[0],language[1])

l 2
h 1
